In [0]:
sc.version

Out[34]: '3.0.1'

In [0]:
#install required python libraries and load them
#%run ./install_python_package
%pip install bs4
%pip install requests
%pip install wget
%pip install pyspark

Python interpreter will be restarted.
Collecting bs4
 Downloading bs4-0.0.1.tar.gz (1.1 kB)
Collecting beautifulsoup4
 Downloading beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
Collecting soupsieve>1.2
 Downloading soupsieve-2.2.1-py3-none-any.whl (33 kB)
Building wheels for collected packages: bs4
 Building wheel for bs4 (setup.py): started
 Building wheel for bs4 (setup.py): finished with status 'done'
 Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=be0dae3c7f669efbac4ed593a57a2672ab22e3cd27884ecf000b11fb4b142afc
 Stored in directory: /home/root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Installing collected packages: soupsieve, beautifulsoup4, bs4
Successfully installed beautifulsoup4-4.9.3 bs4-0.0.1 soupsieve-2.2.1
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: requests in /databricks/python3/lib/python3.7/site-packages (2.22.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests) (2020.6.20)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests) (2.8)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests) (1.25.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests) (3.0.4)
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting wget
 Downloading wget-3.2.zip (10 kB)
Building wheels for collected packages: wget
 Building wheel for wget (setup.py): started
 Building wheel for wget (setup.py): finished with status 'done'
 Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=4fb1dada27d9f18d3fe6d3c6bfbf71fd6aa2e840e9d24f5bf64bfd573c0d9e01
 Stored in directory: /home/root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Installing collected packages: wget
Successfully installed wget-3.2
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting pyspark
 Downloading pyspark-3.1.1.tar.gz (212.3 MB)
Collecting py4j==0.10.9
 Downloading py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Building wheels for collected packages: pyspark
 Building wheel for pyspark (setup.py): started
 Building wheel for pyspark (setup.py): finished with status 'done'
 Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=a27b7dc80b802959e95baddd995e65977b7c67fd9c25c3f52c62e89ac9640fb8
 Stored in directory: /home/root/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark
Installing collected packages: py4j, pyspark
Successfully installed py4j-0.10.9 pyspark-3.1.1
Python interpreter will be restarted.

In [0]:
import numpy as np
import pandas as pd
import requests
import zipfile
from bs4 import BeautifulSoup
import os
import requests
from os.path import basename
import gzip
import shutil
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.types as st
from pyspark.sql.types import StructType,StructField, StringType

In [0]:
### Collecting all compressed file download links
def get_download_links(rooturl, key):
    zip_file_urls = []
    url_address = rooturl
    source_code = requests.get(url_address)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, "html.parser")
    extension = 'gz'

    for link in soup.findAll('a'):
        sub_url = link.get('href')
        if sub_url.endswith(extension):
            sub_url = url_address + sub_url +    key
            zip_file_urls.append(sub_url)
    
    zip_file_df = pd.DataFrame({"url_link": zip_file_urls})
    spark_df = sqlContext.createDataFrame(zip_file_df)
    return(spark_df)
  
    

In [0]:
##[FileInfo(path='dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/', name='Bitcoin-cash/', size=0)]
dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv")


Out[261]: [FileInfo(path='dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv/_committed_305472825023999063', name='_committed_305472825023999063', size=112),
 FileInfo(path='dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv/_started_305472825023999063', name='_started_305472825023999063', size=0),
 FileInfo(path='dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv/part-00000-tid-305472825023999063-d09a1221-8ee6-4084-b9df-16c2137ce200-150-1-c000.csv', name='part-00000-tid-305472825023999063-d09a1221-8ee6-4084-b9df-16c2137ce200-150-1-c000.csv', size=507309)]

In [0]:
##%fs rm -r "/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv"

res16: Boolean = true

In [0]:
## Get compressed file links for Bitcoin-Cash Block data       
def get_file_url_links_spark(url_file_path, root_url, key):          
  url_schema_load = st.StructType([st.StructField('url_link', st.StringType(), True)])
  try:
    spark_url_df = spark.read.csv(url_file_path + "url_list.csv", schema= url_schema_load)
    print(spark_url_df.count())
    if spark_url_df.count() > 0:
      return (spark_url_df)
  except:
    print("Url file not found")
    
  spark_url_df = get_download_links(root_url, key)
  spark_url_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(url_file_path + "url_list.csv")
  return spark_url_df


In [0]:
def get_bitcoin_cash_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/Bitcoin-cash/"
  key = '?key=202001ZjMvj8R3BF'
  bc_cash_blocks_root = 'https://gz.blockchair.com/bitcoin-cash/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, bc_cash_blocks_root, key)  
  return spark_url_df

In [0]:
dl_success = sc.accumulator(0)
dl_fail = sc.accumulator(0)
dl_skip = sc.accumulator(0)

In [0]:
def downloader_new(row):
    zip_file_name = ""
    log_file = open("/databricks/driver/ZipDownloads/download_failures.txt", "a")
    download_success= True
    try:
        # Download and write to file.
        zip_url = row["url_link"]
        resp = requests.get(zip_url)
        key_idx = basename(resp.url).find("?key")
        zip_file_name = basename(resp.url)[:key_idx]        
        #print(f"the filename is {zip_file_name}")
        zip_file = open("ZipDownloads/"+zip_file_name, 'wb')
        zip_file.write(resp.content)  
        zip_file.close()      
        extract_TSV_File(zip_file_name)
        dl_success.add(1)  
        download_failed = False
        
    except Exception as ex:
      print (ex)
      download_success = False
      dl_fail.add(1)
    finally:
      if download_success:
         log_file.write(zip_file_name+","+"0" "\n")
      else:
        log_file.write(zip_file_name+","+"1" "\n")
        
      log_file.close()
                

In [0]:
def extract_TSV_File(ZipFileName):  
  block_size=65536
  log_file = open("/databricks/driver/Extracts/ZipDownloads/Extracts/Extract_failures.txt", "a")
  extract_success= True
  ext = os.path.splitext(ZipFileName)
  dest_file_name = ext[0]
  ext= ext[1]
  if (ext == ".gz"): 
    try:
      dest_file_name_full = "ZipDownloads/Extracts/"+ dest_file_name
      with (gzip.open("ZipDownloads/"+ZipFileName, 'rb')) as s_file, \
        open(dest_file_name_full, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)
    except Exception as ex:
      print("Error in extracting")
      extract_success = False
      print(ex)
    finally:
      if extract_success:
         log_file.write(dest_file_name+","+"0" "\n")
      else:
        log_file.write(dest_file_name+","+"1" "\n")
        
      log_file.close()
      

In [0]:
###Extract tsv files from gz compressed files
def extract_load_into_dataframe(file_directory, bitcastdf):  
  block_size=65536
  for file_name in os.listdir(file_directory): #"/databricks/driver/"):
    ext = os.path.splitext(file_name)
    dest_file_name = ext[0]
    ext= ext[1]
  if (ext == ".gz"): 
    try:
      with (gzip.open(file_name, 'rb')) as s_file, \
        open(dest_file_name, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)
        os.remove("/databricks/driver/" + file_name) 
        tsvdf = pd.read_csv(dest_file_name, sep='\t',  lineterminator='\n', names=None)
        bitcastdf = bitcastdf.append(tsvdf)
    except:
      print("Error in extracting")

In [0]:
def create_bc_block_data_folders():
  ### Down load bitcoin cash block level data
  dbutils.fs.mkdirs("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/")
  dbutils.fs.mkdirs("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData")
  dbutils.fs.mkdirs("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles")
  destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"
  return destlocation_tsv

In [0]:
def create_download_folders():
  dbutils.fs.mkdirs("file:/databricks/driver/ZipDownloads")
  dbutils.fs.mkdirs("file:/databricks/driver/ZipDownloads/Extracts")

In [0]:
def cleanup_driver_location():
  dbutils.fs.rm("file:/databricks/driver/ZipDownloads/",  True)
  dbutils.fs.rm("file:/databricks/driver/ZipDownloads/Extracts/",  True)

In [0]:
### Create sql table
#spark_url_df.createOrReplaceTempView("BC_Block_Url")
q = """select *
from BC_Block_Url limit 10
"""
sub_urldf = spark.sql(q)
sub_urldf.show(2)

+--------------------+
 url_link|
+--------------------+
 url_link|
https://gz.blockc...|
+--------------------+
only showing top 2 rows

In [0]:
### setup download locations in driver
cleanup_driver_location()
create_download_folders()

In [0]:
dbutils.fs.ls("file:/databricks/driver/ZipDownloads/")

Out[169]: [FileInfo(path='file:/databricks/driver/ZipDownloads/Extracts/', name='Extracts/', size=4096)]

In [0]:
### Create structure at mnt for 
create_bc_block_data_folders()

4451

In [0]:
### Get url links to downloade compressed files
spark_url_df = get_bitcoin_cash_blocks_file_url_links()

4451

In [0]:
### Execute download command 
def download_files_in_url_df(url_df):
  %time url_df.foreach(lambda r: downloader_new(r))
  [dl_success.value, dl_skip.value, dl_fail.value]

In [0]:
### Move downloaed compressed and tsv files from driver to DBFS folders
def movedownloads_bc_block_to_dbfs():
  dbutils.fs.mv("file:/databricks/driver/ZipDownloads/Extracts", "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/", recurse=True)
  dbutils.fs.mv("file:/databricks/driver/ZipDownloads/", "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles", recurse = True)

In [0]:
len(dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles/"))
len(dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData"))

Out[190]: 4451

In [0]:
## Code to debug
df_rows = spark_url_df.limit(10).collect()
downloader_new(df_rows[2])
#fname = df_rows[1]['url_link']
#print(fname)
#[dl_success.value, dl_skip.value, dl_fail.value]
#print(postcode_of_lat_long(df_rows[0]['Lat'],df_rows[0]['Long_']))

In [0]:
### Load Extracted TSV Into a DataFrame
def Load_TSV_into_dataframe(dbfs_file_path):
  distFile = sc.textFile(dbfs_file_path)
  return (distFile)
    
    

In [0]:
tsv_files_path = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/blockchair_bitcoin-cash_blocks_*.tsv"


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-215222084237599> in <module> 
 1 tsv_files_path = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/blockchair_bitcoin-cash_blocks_*.csv" 
 2 return_val = Load_TSV_into_dataframe ( tsv_files_path ) 
 ----> 3 return_val . collect ( ) 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 901 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 902 with SCCallSiteSync ( self . context ) as css : 
 --> 903 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 904 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 905 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input Pattern dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/blockchair_bitcoin-cash_blocks_*.csv matches 0 files
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:213)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:283)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:279)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:57)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:283)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:279)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2398)
	at org.apache.spark.rdd.RDD.$anonfun$collect$1(RDD.scala:1011)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:125)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:395)
	at org.apache.spark.rdd.RDD.collect(RDD.scala:1010)
	at org.apache.spark.api.python.PythonRDD$.collectAndServe(PythonRDD.scala:260)
	at org.apache.spark.api.python.PythonRDD.collectAndServe(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles/download_failures.txt")
extract_df = spark.read.csv("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles/download_failures.txt")
### Create sql table
extract_df.createOrReplaceTempView("extract_log_table")


In [0]:
row_data = dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/")
tsv_file_path = [x[0] for x in row_data]
tsv_file_df = spark.createDataFrame(tsv_file_path, StringType())

In [0]:
tsv_file_df = spark.createDataFrame(tsv_file_path, StringType())

+--------------------+
 value|
+--------------------+
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
dbfs:/mnt/BlockCh...|
+--------------------+
only showing top 10 rows

In [0]:
bc_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/*.tsv")
bc_block_tsv_rdd_Df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/*.tsv")


In [0]:
bc_block_tsv_rdd_Df.count()

Out[2]: 678886

In [0]:
bc_block_tsv_rdd_Df.columns

Out[3]: ['id',
 'hash',
 'time',
 'median_time',
 'size',
 'version',
 'version_hex',
 'version_bits',
 'merkle_root',
 'nonce',
 'bits',
 'difficulty',
 'chainwork',
 'coinbase_data_hex',
 'transaction_count',
 'input_count',
 'output_count',
 'input_total',
 'input_total_usd',
 'output_total',
 'output_total_usd',
 'fee_total',
 'fee_total_usd',
 'fee_per_kb',
 'fee_per_kb_usd',
 'cdd_total',
 'generation',
 'generation_usd',
 'reward',
 'reward_usd',
 'guessed_miner']

In [0]:
## create a csv fiel with subset of columns
bc_block_tsv_rdd_Df[["ID", "time","size","median_time","guessed_miner","difficulty","fee_total", "fee_total_usd", "reward", "reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv")

In [0]:
#dbutils.fs.rm("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv", recurse = True)

Out[16]: True

In [0]:
#databricks fs cp  
## copy to filestore to download the file using web browser
dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv", "dbfs:/FileStore/bc_block_sub_df_exported.csv", recurse = True)

Out[20]: True

In [0]:
bc_block_tsv_rdd_Df[["guessed_miner"]].collect().unique()["guessed_miner"]

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3370335093029502> in <module> 
 ----> 1 bc_block_tsv_rdd_Df [ [ "guessed_miner" ] ] . collect ( ) . unique ( ) [ "guessed_miner" ] 

 AttributeError : 'list' object has no attribute 'unique'